<a href="https://colab.research.google.com/github/sv9-ru/flow_meter/blob/main/%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C1_2(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#для скачивания моделей pkl
!git clone https://github.com/sv9-ru/flow_meter.git

Cloning into 'flow_meter'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 6), reused 7 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 1.34 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
%cd /content/flow_meter
!find . -maxdepth 1 -name "*.pkl" -exec cp {} /content/ \;
%cd /content
!echo "Копирование завершено. Файлы .pkl теперь в /content"
!ls /content/*.pkl

/content/flow_meter
/content
Копирование завершено. Файлы .pkl теперь в /content
/content/best_model_density.pkl  /content/best_model_mass_flow.pkl


In [3]:
%%writefile cor.py
import pandas as pd
import pickle
import time
import os
import numpy as np
from scipy.spatial import KDTree

# 1. Определение класса (обязательно в начале файла)
class CoriolisModel:
    def __init__(self, config):
        self.config = config
        self.type = config['type']
        self.X_train = config['X_train']
        self.y_train = config['y_train']
        self.scaler = config['scaler']
        self.k = config.get('k')
        self.h = config.get('h')
        self.kernel = config.get('kernel', 'gaussian')
        if 'kdtree' in self.type:
            self.tree = KDTree(self.X_train)

    def _kernel_weights(self, dist, h, kernel_type):
        u = dist / h
        if kernel_type == 'rectangular': return np.where(np.abs(u) <= 1, 1.0, 0.0)
        elif kernel_type == 'triangular': return np.where(np.abs(u) <= 1, 1 - np.abs(u), 0.0)
        elif kernel_type == 'gaussian': return np.exp(-0.5 * u**2) / np.sqrt(2 * np.pi)
        return np.ones_like(dist)

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        X_scaled = self.scaler.transform(X)
        preds = []
        for x in X_scaled:
            if self.type == 'kdtree_weighted':
                dists, idx = self.tree.query(x, k=self.k)
                w = self._kernel_weights(dists, self.h, self.kernel)
                pred = np.sum(w * self.y_train[idx]) / (np.sum(w) + 1e-12)
            elif self.type == 'weighted_knn_gaussian':
                dists = np.linalg.norm(self.X_train - x, axis=1)
                weights = np.exp(-0.5 * (dists / self.h) ** 2)
                idx = np.argsort(dists)[:self.k]
                pred = np.sum(weights[idx] * self.y_train[idx]) / (np.sum(weights[idx]) + 1e-12)
            elif self.type == 'weighted_knn':
                dists = np.linalg.norm(self.X_train - x, axis=1)
                idx = np.argsort(dists)[:self.k]
                w = 1 / (dists[idx] + 1e-6)
                pred = np.sum(self.y_train[idx] * w) / np.sum(w)
            elif self.type == 'kdtree_knn':
                dists, idx = self.tree.query(x, k=self.k)
                pred = np.mean(self.y_train[idx])
            else:
                dists = np.linalg.norm(self.X_train - x, axis=1)
                idx = np.argsort(dists)[:self.k]
                pred = np.mean(self.y_train[idx])
            preds.append(pred)
        return np.array(preds)

# 2. Функция загрузки моделей
def load_models():
    """Загружает модели из текущей директории."""
    file_mf = 'best_model_mass_flow.pkl'
    file_rho = 'best_model_density.pkl'

    if not os.path.exists(file_mf) or not os.path.exists(file_rho):
        print(f"Ошибка: файлы {file_mf} или {file_rho} не найдены.")
        return None, None

    try:
        with open(file_mf, 'rb') as f:
            m_flow = pickle.load(f)
        with open(file_rho, 'rb') as f:
            m_density = pickle.load(f)
        return m_flow, m_density
    except Exception as e:
        print(f"Ошибка загрузки через pickle: {e}")
        return None, None

# 3. Функция для предсказания (используется внешними скриптами)
def predict_correction(flow_input, density_drop_input, model_flow, model_density):
    input_df = pd.DataFrame([[flow_input, density_drop_input]],
                            columns=['Cor Mass Flow', 'Cor Density Drop'])

    start_time = time.perf_counter()

    # Предсказание % ошибки
    err_f = model_flow.predict(input_df)[0]
    err_d = model_density.predict(input_df)[0]

    # Применение формулы коррекции
    corrected_f = flow_input * (1 - err_f / 100)
    corrected_d = density_drop_input * (1 - err_d / 100)

    execution_time = time.perf_counter() - start_time
    return corrected_f, corrected_d, execution_time

# 4. Логика интерактивного запуска (main)
def main():
    m_flow, m_density = load_models()
    if not m_flow or not m_density:
        return

    print("\n--- Программа коррекции Coriolis v1.0 ---")
    try:
        f_in = float(input("Введите расход (Cor Mass Flow): "))
        d_in = float(input("Введите падение плотности (Cor Density Drop): "))

        res_f, res_d, exec_time = predict_correction(f_in, d_in, m_flow, m_density)

        print("-" * 30)
        print(f"Скорректированный расход: {res_f:.6f}")
        print(f"Скорректированная плотность: {res_d:.6f}")
        print(f"Время выполнения: {exec_time:.6f} сек.")
        print("-" * 30)

    except ValueError:
        print("Ошибка: вводите только числовые значения.")

if __name__ == "__main__":
    main()

Writing cor.py


In [4]:
%%writefile run_cor.py
import time
import pandas as pd
from cor import load_models, predict_correction, CoriolisModel

def main():
    # Название файла с данными
    input_csv = 'test_data.csv'

    print("Загрузка моделей...")
    m_flow, m_density = load_models()

    if m_flow is None:
        print("Ошибка: Модели не загружены.")
        return

    # Проверка наличия файла данных
    try:
        df_test = pd.read_csv(input_csv)
        print(f"Файл {input_csv} успешно загружен. Найдено точек: {len(df_test)}")
    except FileNotFoundError:
        print(f"Ошибка: Файл {input_csv} не найден в текущей директории.")
        return

    print("\n" + "="*85)
    print(f"{'№':<3} | {'Вход Flow/Drop':<22} | {'Выход Flow/Dens':<25} | {'Время (сек)':<12}")
    print("-" * 85)

    total_start_time = time.perf_counter()

    # Итерируемся по строкам CSV
    for i, row in df_test.iterrows():
        f_in = row['flow']
        d_in = row['density_drop']

        # Выполняем коррекцию
        res_f, res_d, single_time = predict_correction(f_in, d_in, m_flow, m_density)

        print(f"{i+1:<3} | {f_in:>9.4f} / {d_in:>8.4f} | {res_f:>10.5f} / {res_d:>10.5f} | {single_time:.6f}")

    total_duration = time.perf_counter() - total_start_time

    print("-" * 85)
    print(f"ОБЩЕЕ ВРЕМЯ ОБРАБОТКИ ТАБЛИЦЫ: {total_duration:.6f} сек.")
    print(f"СРЕДНЕЕ ВРЕМЯ НА ОДНУ СТРОКУ: {total_duration/len(df_test):.6f} сек.")
    print("="*85)

if __name__ == "__main__":
    main()

Writing run_cor.py


In [5]:
%%writefile test_data.csv
flow,density_drop
0.382319,-0.041438
0.372571,1.272146
0.374177,3.202944
0.349452,6.324710
3.055405,28.584310

Writing test_data.csv


In [6]:
!python /content/cor.py


--- Программа коррекции Coriolis v1.0 ---
Введите расход (Cor Mass Flow): Traceback (most recent call last):
  File "/content/cor.py", line 120, in <module>
    main()
  File "/content/cor.py", line 105, in main
    f_in = float(input("Введите расход (Cor Mass Flow): "))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
^C


In [7]:
!python /content/run_cor.py

Загрузка моделей...
Файл test_data.csv успешно загружен. Найдено точек: 5

№   | Вход Flow/Drop         | Выход Flow/Dens           | Время (сек) 
-------------------------------------------------------------------------------------
1   |    0.3823 /  -0.0414 |    0.39795 /   -0.04097 | 0.009725
2   |    0.3726 /   1.2721 |    0.39306 /    1.25674 | 0.000887
3   |    0.3742 /   3.2029 |    0.39476 /    3.14250 | 0.000720
4   |    0.3495 /   6.3247 |    0.37485 /    6.07019 | 0.000780
5   |    3.0554 /  28.5843 |    3.20304 /   27.64436 | 0.000607
-------------------------------------------------------------------------------------
ОБЩЕЕ ВРЕМЯ ОБРАБОТКИ ТАБЛИЦЫ: 0.021967 сек.
СРЕДНЕЕ ВРЕМЯ НА ОДНУ СТРОКУ: 0.004393 сек.


In [8]:
# 1. Упаковываем все нужные файлы в один zip-архив
!zip -r my_project.zip /content/cor.py /content/run_cor.py /content/test_data.csv *.pkl

# 2. Скачиваем полученный архив
from google.colab import files
files.download('my_project.zip')

  adding: content/cor.py (deflated 64%)
  adding: content/run_cor.py (deflated 50%)
  adding: content/test_data.csv (deflated 19%)
  adding: best_model_density.pkl (deflated 52%)
  adding: best_model_mass_flow.pkl (deflated 11%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>